# Analyse data for line loop navigation coming from the homecage 

In [5]:
%matplotlib inline

import numpy as np
import os
import sys
import re
import matplotlib.pyplot as plt

## Admin to get hold of all the subject folders 

In [41]:
experiment_folders = [r"C:/Users/takam/Desktop/Dropbox/pyhomecage/data/ec_lineloop_1",
                     r"C:/Users/takam/Desktop/Dropbox/pyhomecage/data/ec_lineloop_2"]

subject_folders = []
for ef in experiment_folders:
    sf1 = os.listdir(ef)
    
    for sf_ in sf1:
        sf = os.path.join(ef,sf_)
        subject_folders.append(os.path.join(sf,os.listdir(sf)[0]))
    

##  Main function to analyse probe trial performance

In [42]:
def get_probe_trial_performance_homecage(lines,probe_dict={}):
    """ """
    times = []
    probe_perf = []
    target_hist = []; probe_hist = []; poked_hist = []; dir_hist = []; seq_hist = []
    #if probe_dict is None: 
    #    probe_dict = {}
    for ix,l_ in enumerate(lines):
        #if 'task_number' in l_:
            #print()
            #print(l_)
            #print()
        if 'TARGET:' in l_:
            t_ = re.findall('P ([0-9]*) R',l_)[0]
            times.append(float(t_))
            probe = eval(re.findall('PROBE: (False|True)',l_)[0])
            target = int(re.findall('TARGET: ([0-9]*)',l_)[0])
            poked = int(re.findall('POKED: ([0-9]*)',l_)[0])
            direction = int(re.findall('DIR: (.*)',l_)[0])
            seq_ix = int(re.findall('SEQ_IX: ([0-9]*)',l_)[0])
            if seq_hist:
                if probe:
                    #print(target)
                    if not (probe_hist[-1] and (target==target_hist[-1])):
                        correct_poke = poked==target
                        probe_perf.append(correct_poke)
                        transition = str(target_hist[-1]) + '_' + str(target)
                        #print(probe_dict.keys())
                        if transition in probe_dict.keys():
                            probe_dict[transition].append(correct_poke)
                        else:
                            probe_dict[transition] = [correct_poke]
                        
            target_hist.append(target); probe_hist.append(probe); dir_hist.append(direction);
            poked_hist.append(poked); seq_hist.append(seq_ix)
    #print(len(probe_perf))
    return probe_dict

##  Loop analysis over all subjects

In [43]:
subject_folders = [r"C:/Users/takam/Desktop/Dropbox/pyhomecage/data/ec_lineloop_1\C00494062_5\16190096773636293_task"]

SyntaxError: unexpected EOF while parsing (<ipython-input-43-d574b8ca4480>, line 1)

In [40]:
for ROOT in subject_folders:
    print(ROOT)
    fs = [i for i in os.listdir(ROOT) if '_taskFile' not in i]
    probe_dict = {}
    for f_ in fs:
        if 'follow_the_light_2_graphs' in f_:
            #print(f_)
            fpath = os.path.join(ROOT,f_)
            with open(fpath,'r') as f:
                lines = f.readlines()

            probe_dict = get_probe_trial_performance_homecage(lines,probe_dict=probe_dict)
            #print(probe_dict)
            
    print(len(probe_dict))
    for k,v in probe_dict.items():
        print(k,np.mean(v))

C:/Users/takam/Desktop/Dropbox/pyhomecage/data/ec_lineloop_1\C00494062_5\16190096773636293_task
4
8_4 0.23076923076923078
2_4 0.7333333333333333
6_3 0.5
1_3 0.0


In [35]:
len(probe_dict)

4

In [177]:
ROOT = r"C:/Users/takam/Desktop/Dropbox/pyhomecage/data/ec_lineloop_1/C00494062_1/16190095855355382_task/"

fs = [i for i in os.listdir(ROOT) if '_taskFile' not in i]

In [138]:
#fpath = r'C:/Users/takam/Desktop/Dropbox/pyhomecage/data/ec_lineloop_2/C00499495_6/16201231547824955_task/C00499495_6_ec_lineloop_2_follow_the_light_2_graphs'

In [139]:
probe_dict = {}
for f_ in fs:
    if 'follow_the_light_2_graphs' in f_:
        #print(f_)
        fpath = os.path.join(ROOT,f_)
        with open(fpath,'r') as f:
            lines = f.readlines()

        task_lines = []
        for l_ in lines:
            if 'POKED' in l_:
                task_lines.append(l_)
        probe_dict = get_probe_trial_performance_homecage(task_lines,probe_dict=probe_dict)
        #print(probe_dict)

26
0
13


In [150]:
def get_probe_trial_performance_homecage(lines,probe_dict={}):
    """ """
    times = []
    probe_perf = []
    target_hist = []; probe_hist = []; poked_hist = []; dir_hist = []; seq_hist = []
    #if probe_dict is None: 
    #    probe_dict = {}
    for ix,l_ in enumerate(lines):
        
        if 'TARGET' in l_:
            t_ = re.findall('P ([0-9]*) R',l_)[0]
            times.append(float(t_))
            probe = eval(re.findall('PROBE: (False|True)',l_)[0])
            target = int(re.findall('TARGET: ([0-9]*)',l_)[0])
            poked = int(re.findall('POKED: ([0-9]*)',l_)[0])
            direction = int(re.findall('DIR: (.*)',l_)[0])
            seq_ix = int(re.findall('SEQ_IX: ([0-9]*)',l_)[0])

            if probe:
                if not (probe_hist[-1] and (target==target_hist[-1])):
                    correct_poke = poked==target
                    probe_perf.append(correct_poke)
                    transition = str(seq_hist[-1]) + '_' + str(seq_ix)
                    if transition in probe_dict.keys():
                        probe_dict[transition].append(correct_poke)
                    else:
                        probe_dict[transition] = [correct_poke]
            target_hist.append(target); probe_hist.append(probe); dir_hist.append(direction);
            poked_hist.append(poked); seq_hist.append(seq_ix)
    #print(len(probe_perf))
    return probe_dict

In [ ]:
 A B C

In [13]:
l_

'P 146 REWS:0, POKED: 4, PROBE: False , TARGET: 6,SEQ_IX: 1, DIR: 1\n'